In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
# Importing external packages
import open3d as o3d

In [16]:
# Importing internal packages
from src.features.point_cloud_handler import Handler
from src.features.pre_process import Reduce
from src.enums import Files
from src.features.cluster import PointCloudDBSCAN
from src.features.slicer import Slicer
from src.features.ransac import RANSACCylinderFitting
from src.features.display import Visualize
from src.features.stem_detector import StemDetector

In [17]:
raw_point_cloud: o3d.geometry.PointCloud = Handler.open(Files.TREE_POINT_CLOUD)

Loaded 17452822 points from tree_point_cloud.las


In [18]:
down_sampled_point_cloud: o3d.geometry.PointCloud = Reduce.voxel_downsample(
    point_cloud=raw_point_cloud
)

Original point count: 17452822
Downsampled point count: 1248345
Point cloud size reduced with 92.85%


In [19]:
slicer: Slicer = Slicer(point_cloud=down_sampled_point_cloud)

In [20]:
trees: o3d.geometry.PointCloud = o3d.geometry.PointCloud()

for i in range(len(slicer.point_cloud_slices)):
    sliced_point_cloud = slicer.point_cloud_slices[i]
    dbscan: PointCloudDBSCAN = PointCloudDBSCAN(point_cloud=sliced_point_cloud)
    dbscan.cluster()
    ransac: RANSACCylinderFitting = RANSACCylinderFitting(
        point_cloud=sliced_point_cloud,
        dbscan=dbscan
    )
    
    trees += ransac.trees
    del dbscan
    del ransac

Point cloud has 25 clusters
Point cloud has 65 clusters
Point cloud has 130 clusters
Point cloud has 160 clusters
Point cloud has 245 clusters
Point cloud has 374 clusters
Point cloud has 447 clusters


In [21]:
trees

PointCloud with 6573 points.

In [22]:
Reduce.statistical_outlier_removal(
    point_cloud=trees,
    nb_neighbors=20,
    std_ratio=.1
)

PointCloud with 4891 points.

In [23]:
dbscan: 'PointCloudDBSCAN' = PointCloudDBSCAN(
    point_cloud=trees,
    eps=1.2,
    min_points=10
)

dbscan.cluster()

Point cloud has 185 clusters


In [24]:
stem_detector: 'StemDetector' = StemDetector(
    trees=trees,
    dbscan=dbscan
)

Visualize.display_colored_clusters(trees, stem_detector.dbscan.labels)

In [25]:
stems: list[o3d.geometry.AxisAlignedBoundingBox] = stem_detector.stems
trees: list[o3d.geometry.PointCloud] = []
for i in range(len(stems)):
    if i == 0:
        continue
        
    stem = stems[i]
    tree = down_sampled_point_cloud.crop(stem)
    trees.append(tree)
    

In [27]:
Visualize.display(*trees)

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Ugyldig referanse. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Forespurt transformeringsoperasjon er ikke støttet. 
